# 1 自動收集

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
import time

driver = webdriver.Firefox()  
url = "http://dh.ersjk.com/jsp/front/prodlist.jsp"
driver.get(url)

In [ ]:
rname = '' # 賬號
rpass = '' # 密碼

In [2]:
driver.find_element(By.ID, "rname").click()
driver.find_element(By.ID, "rname").send_keys(rname)
driver.find_element(By.ID, "rpass").click()
driver.find_element(By.ID, "rpass").send_keys(rpass)
driver.find_element(By.LINK_TEXT, "登 录").click()

In [3]:
driver.find_element(By.CSS_SELECTOR, ".button:nth-child(1)").click()
driver.find_element(By.ID, "selInput").click()
driver.find_element(By.CSS_SELECTOR, "#jsfw > img").click()
driver.find_element(By.CSS_SELECTOR, ".button:nth-child(2)").click()
driver.find_element(By.CSS_SELECTOR, ".close").click()

In [4]:
def scrape_data_to_excel(keyword,file_path):
    
    #輸入檢索內容
    input_keyword = driver.find_element(By.XPATH, '//*[@id="ikeyword"]')
    input_keyword.clear()
    input_keyword.send_keys(keyword)
    driver.find_element(By.XPATH, '//*[@id="iglju"]/button').click()
    time.sleep(5)
    
    #循環爬取頁面內容
    df = pd.DataFrame()
    page_num = driver.find_element(By.XPATH, '//*[@id="pagebox"]/tbody/tr/td[1]/span[2]').text
    n = int(''.join(filter(str.isdigit, page_num)))
    lis = driver.find_elements(By.XPATH, '//*[@id="list_tb"]/li')
    if lis:
        for i in range(n):
            lis = driver.find_elements(By.XPATH, '//*[@id="list_tb"]/li')
            for li in lis:
                bname = li.find_element(By.XPATH, './div[1]').text
                volume = li.find_element(By.XPATH, './div[2]').text
                text = li.find_element(By.XPATH, './div[3]').text
                word = li.find_element(By.XPATH, './div[3]/span[1]').text
                df = pd.concat([df, pd.DataFrame({'書名': [bname], '卷': [volume], '文本': [text], '詞彙': [word]})], ignore_index=True)
            try:
                next_page = driver.find_element(By.XPATH,'//*[@id="pagebox"]/tbody/tr/td[1]/a[last()]')
                next_page.click()
            except NoSuchElementException:
                pass #如果僅有一頁，就直接跳過點擊下一頁步驟

            time.sleep(7)

        #刪除文本中的雙引號
        df['文本'] = df['文本'].str.replace('“', '')
        df['文本'] = df['文本'].str.replace('”', '')
        df = df.drop_duplicates() #刪除重複

        df.to_excel(file_path, index=False)
    else:
        pass

In [6]:
keywords = '秋老伏、進霑'.split('、')

folder_path = r'F:\古籍處理數據\input\繪圖'
for keyword in keywords:
    file_path = f'{folder_path}/{keyword}.xlsx'
    scrape_data_to_excel(keyword,file_path)

# 2 處理文本內容

標點、分詞、添加時空信息和提示並分類

In [21]:
import torch
from transformers import pipeline
import pandas as pd
import datasets
import re
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pipe = pipeline("token-classification", model=r"E:\\jupyter_notebook\\classical-chinese-punctuation-guwen-biaodian", device=device)
pipe2 = pipeline("token-classification", model=r"E:\\jupyter_notebook\\bert-base-chinese-ws", device=device)

In [22]:
# 定義標點函數
def punctuate_text(sample, batch_mode=False):
    if batch_mode:
        texts = sample['文本']
    else:
        texts = [sample['文本']]

    punctuated_texts = pipe(texts)

    def process_single_text(punctuated_text, original_text):
        punctuation_dict = {p['start']: p['entity'] for p in punctuated_text}
        punctuated = "".join(original_text[i] + punctuation_dict.get(i, '') for i in range(len(original_text)))
        return punctuated

    processed_texts = [process_single_text(punctuated_text, text) for punctuated_text, text in zip(punctuated_texts, texts)]
    if batch_mode:
        return {'文本': processed_texts}
    else:
        return {'文本': processed_texts[0]}

# 定義分詞函數
def word_segmentation(sample, batch_mode=False):
    if batch_mode:
        texts = sample['文本']
    else:
        texts = [sample['文本']]

    results = pipe2(texts)

    def process_single_text(result):
        processed_text = ''.join([' ' + entry['word']
                                  if entry['entity'] == 'B'
                                  else entry['word']
                                  for entry in result])
        return processed_text.strip()

    processed_texts = [process_single_text(result) for result in results]

    if batch_mode:
        return {'分詞': processed_texts}
    else:
        return {'分詞': processed_texts[0]}

In [23]:
def process_xlsx_file(file_path):
    print(f'正在處理的文件是{file_path}')
    df = pd.read_excel(file_path)

    # 讀取爲datasets
    ds = datasets.Dataset.from_pandas(df)

    batch_mode = torch.cuda.is_available()
    if batch_mode:
        print('正在標點...')
        batch_size = 16384
        def add_punctuate_text(sample):
            return punctuate_text(sample, batch_mode=True)
        ds = ds.map(add_punctuate_text, batched=True, batch_size=batch_size)
        print('正在分詞...')
        def add_word_segmentation(sample):
            return word_segmentation(sample, batch_mode=True)
        ds = ds.map(add_word_segmentation, batched=True, batch_size=batch_size)

    else:
        print('正在標點...')
        def add_punctuate_text_single(sample):
            return punctuate_text(sample)
        ds = ds.map(add_punctuate_text_single)
        print('正在分詞...')
        def add_word_segmentation_single(sample):
            return word_segmentation(sample)
        ds = ds.map(add_word_segmentation_single)

    # 轉換回pandas DataFrame
    df = ds.to_pandas()

    # 添加時空信息
    print('正在添加時空信息...')
    df_1 = pd.read_excel(r'E:\坚果云同步文件\论文内容\博士階段論文\畢業論文\論文數據\愛如生數據庫書目（合併重複書名項、添加時代、索引年、坐標）.xlsx')

    df1_dict = df_1.set_index('書名')[['時代作者', '版本', '索引年', '時間段', '地域','X', 'Y', 'sys_id', 'uri']].to_dict(orient='index')

    def get_infor(x):
        return df1_dict.get(x, {'時代作者': None, '版本': None, '索引年': None,'時間段': None, '地域': None, 'X': None, 'Y': None,  'sys_id': None, 'uri': None})

    df[['時代作者', '版本', '索引年', '時間段', '地域','X', 'Y', 'sys_id', 'uri']] = df['書名'].apply(get_infor).apply(pd.Series)

    #添加提示列
    df['詞彙'] = df['詞彙'].str.replace(' ', '') # 刪除空格
    keywords = df['詞彙'].unique().tolist() # 提取關鍵詞
    spaced_keywords = []
    for keyword in keywords: # 給關鍵詞添加空格
        spaced_keyword = ' '.join(keyword)
        spaced_keyword = ' ' + spaced_keyword + ' '
        spaced_keywords.append(spaced_keyword)
    keywords += spaced_keywords

    df['提示'] = '可能非所需的詞彙'
    for text in df['分詞']:
        for keyword in keywords:
            if keyword in text:
                df.loc[df['分詞'] == text, '提示'] = None

    # 用<span></span>標籤括住“文本”列中的關鍵詞
    for keyword in keywords:
        pattern = re.compile(re.escape(keyword))
        df['文本'] = df['文本'].apply(lambda x: pattern.sub(f'<span>{keyword}</span>', x))

    df = df.drop(['詞彙'], axis=1)
    df.to_excel(file_path, index=False)
    print(f'文件已經保存在{file_path}')
    return keywords # 返回keywords，以便後續使用

In [24]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

def stopwords_list(file_path, keywords):
    '''
    file_path: 輸入以utf-8編碼的txt文件，文件以回車分隔停用詞;
    keywords：將查詢的詞彙列表也當作停用詞，避免分類的時候受到干擾。
    '''
    with open(file_path, 'r', encoding='utf-8-sig') as f:
        stopwords = f.read().split('\n')
        new_keywords = keywords + list(''.join(keywords))
        stopwords = stopwords + new_keywords + ['\n', '\r','\r\n']
        stopwords = list(set(stopwords))
    return stopwords

def text_clustering(file_path, stop_words):
    '''
    如果語料大於300條，進行聚類以便人工閱讀。聚類數設置爲語料總數的1/50；
    file_path: 需聚類的xlsx文件，其中有“分詞”列，是已分好詞的文本；
    stop_words: 停用詞列表。
    '''
    df = pd.read_excel(file_path)
    corpus = df['分詞']
    
    if len(corpus) <= 300:
        print('語料內容不多，無需聚類。')
    else:
        tfidf_vectorizer = TfidfVectorizer(stop_words=stop_words) # 創建TF-IDF向量化器，同時設置停用詞
        tfidf_matrix = tfidf_vectorizer.fit_transform(corpus) # 將文本數據轉換成TF-IDF特征矩陣
        # 使用K均值聚類
        num_clusters = int(len(corpus) / 50)  # 將聚類數設置爲語料數的1/50
        kmeans = KMeans(n_clusters=num_clusters, n_init='auto', random_state=42)
        kmeans.fit(tfidf_matrix)

        cluster_labels = kmeans.labels_ # 獲取聚類標籤
        df['聚类标签'] = cluster_labels

        #按照“聚類標籤”列的值進行分組，並且分組內按照“文本”列排序
        df_sorted = df.groupby('聚类标签', group_keys=False).apply(lambda x: x.sort_values(by='文本'))
        df_sorted = df_sorted.reset_index(drop=True) # 重置索引以获取最终的排序结果
        
        df_sorted.to_excel(file_path, index=False)
        print('已聚類。')
        

In [26]:
file_path = r'F:\古籍處理數據\input\繪圖\官圍.xlsx'
keywords = process_xlsx_file(file_path)
stop_words = stopwords_list(r'F:\古籍處理數據\input\繪圖\停用詞.txt', keywords)
text_clustering(file_path, stop_words)

正在處理的文件是F:\古籍處理數據\input\繪圖\官圍.xlsx
正在標點...


Map:   0%|          | 0/530 [00:00<?, ? examples/s]

正在分詞...


Map:   0%|          | 0/530 [00:00<?, ? examples/s]

正在添加時空信息...
文件已經保存在F:\古籍處理數據\input\繪圖\官圍.xlsx
已聚類。
